# Module 2 Summative Lab

## The Goal  - Extract, Transform, and Load (ETL)

Upon completion of this lab, each unique team in this dataset should have a record in the MongoDB instance containing the following information:

* The name of the team
* The total number of goals scored by the team during the 2011 season
* The total number of wins the team earned during the 2011 season
* A histogram visualization of the team's wins and losses for the 2011 season (store the visualization directly by assigning it to a variable)
* The team's win percentage on days where it was raining during games in the 2011 season.

***Object-Oriented Programming is a requirement for this project--you must create at least 2 separate, well-structured classes in your solution!***

## Plan of attach
1. Plan the plan
2. Query the SQL database
3. Calculate summary statistics
4. Get the weather data from the DarkSky API
5. Load the data into MongoDB
6. Refactor

Be sure to plan the inputs, outputs, and methods for each class before you begin coding!

## Classes

In [ ]:
class Query_SQL():
    pass

In [ ]:
class Summary_stats():
    pass

In [ ]:
class WeatherGetter():
    pass

In [ ]:
class MongoHandler():
    pass

## Query the Database

In [1]:
# Import sqlite3, load connection and cursor
import sqlite3 
conn = sqlite3.connect('database.sqlite')
cur = conn.cursor()

In [17]:
# Look at the table of Teams (How many teams where there in 2011?)
cur.execute("""SELECT * FROM teams WHERE Season = 2011;""")
import pandas as pd
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
print(len(df))
df.head()

36


,Season,TeamName,KaderHome,AvgAgeHome,ForeignPlayersHome,OverallMarketValueHome,AvgMarketValueHome,StadiumCapacity
0,2011,Bayern Munich,24,25,11,335600000,13980000,75000
1,2011,Dortmund,28,24,12,158200000,5650000,81359
2,2011,Leverkusen,29,23,9,151100000,5210000,30210
3,2011,Schalke 04,37,24,20,136730000,3700000,62271
4,2011,Werder Bremen,38,23,17,125350000,3300000,42100


In [19]:
# Look at the Matches table for same season.
cur.execute("""SELECT * FROM matches WHERE Season = 2011;""")
import pandas as pd
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
print(len(df))  # Total number of matches
df.head()

992


,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A
2,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A
3,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H
4,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D


In [12]:
# Look at the table for Unique_Teams
cur.execute("""SELECT * FROM Unique_Teams;""")
import pandas as pd
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
print(len(df))  # Total number of teams (for all seasons)
df.head()

128


,TeamName,Unique_Team_ID
0,Bayern Munich,1
1,Dortmund,2
2,Leverkusen,3
3,RB Leipzig,4
4,Schalke 04,5


In [38]:
# Look at the table for Teams_in_Matches
cur.execute("""SELECT * FROM Teams_in_Matches;""")
import pandas as pd
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
print(len(df)) 
print(len(df)/2)  # Total Number of Matches (for all seasons)
df.head()

49148
24574.0


,Match_ID,Unique_Team_ID
0,1,26
1,1,46
2,2,26
3,2,42
4,3,26


In [164]:
# Try to join some of the tables
cur.execute("""SELECT * 
               FROM Teams_in_Matches
               JOIN Unique_Teams
               USING(Unique_Team_ID)
               JOIN Matches
               USING(Match_ID)
               WHERE Season = 2011
               ORDER BY Match_ID DESC
               ;""")
import pandas as pd
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
print(len(df))
print(len(df)/2) # Total number of matches for 2011 season
df.head()

1984
992.0


,Match_ID,Unique_Team_ID,TeamName,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,44874,119,Wolves,E0,2011,2012-05-13,Wigan,Wolves,3,2,H
1,44874,120,Wigan,E0,2011,2012-05-13,Wigan,Wolves,3,2,H
2,44873,81,Arsenal,E0,2011,2012-05-13,West Brom,Arsenal,2,3,A
3,44873,117,West Brom,E0,2011,2012-05-13,West Brom,Arsenal,2,3,A
4,44872,92,Tottenham,E0,2011,2012-05-13,Tottenham,Fulham,2,0,H


In [165]:
# Total number of Homewins, Awaywins, and Draws for the 2011 season
df['FTR'].value_counts(normalize=True) * 992

H    450.0
A    279.0
D    263.0
Name: FTR, dtype: float64

In [166]:
# Try to play with the data stats - total home goals and away goals
cur.execute("""SELECT SUM(FTHG) AS Homegoals, SUM(FTAG) AS Awaygoals, SUM(FTHG) + SUM(FTAG) AS Total_Goals
               FROM Teams_in_Matches
               JOIN Unique_Teams
               USING(Unique_Team_ID)
               JOIN Matches
               USING(Match_ID)
               WHERE Season = 2011
               ;""")
import pandas as pd
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
print(len(df))
df.head()
# This seems like too many goals!

1


,Homegoals,Awaygoals,Total_Goals
0,3200,2392,5592


In [167]:
# Try to play with the data stats - home wins?
cur.execute("""SELECT *
               FROM Teams_in_Matches
               JOIN Unique_Teams
               USING(Unique_Team_ID)
               JOIN Matches
               USING(Match_ID)
               WHERE Season = 2011
               GROUP BY Match_ID
               HAVING FTHG > FTAG
               ;""")
import pandas as pd
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
print(len(df))
df.head()

450


,Match_ID,Unique_Team_ID,TeamName,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1095,11,Mainz,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H
1,1097,6,M'gladbach,D1,2011,2012-01-20,M'gladbach,Bayern Munich,3,1,H
2,1101,18,Hannover,D1,2011,2011-10-23,Hannover,Bayern Munich,2,1,H
3,1103,3,Leverkusen,D1,2011,2012-03-03,Leverkusen,Bayern Munich,2,0,H
4,1108,2,Dortmund,D1,2011,2012-04-11,Dortmund,Bayern Munich,1,0,H


In [168]:
# Try to play with the data stats - away wins?
cur.execute("""SELECT Match_ID, AwayTeam, Date, FTR
               FROM Teams_in_Matches
               JOIN Unique_Teams
               USING(Unique_Team_ID)
               JOIN Matches
               USING(Match_ID)
               WHERE Season = 2011
               GROUP BY Match_ID
               HAVING FTHG < FTAG
               ;""")
import pandas as pd
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
print(len(df))
df.head()

279


,Match_ID,AwayTeam,Date,FTR
0,1092,Bayern Munich,2012-03-31,A
1,1093,Bayern Munich,2011-12-11,A
2,1094,Bayern Munich,2011-08-13,A
3,1099,Bayern Munich,2012-04-21,A
4,1100,Bayern Munich,2011-09-18,A


In [169]:
# Try to play with the data stats - team wins?
cur.execute("""SELECT TeamName, HomeTeam, FTR, Date
               FROM Teams_in_Matches
               JOIN Unique_Teams
               USING(Unique_Team_ID)
               JOIN Matches
               USING(Match_ID)
               WHERE Season = 2011
               GROUP BY Match_ID
                
               ;""")
import pandas as pd
df2 = pd.DataFrame(cur.fetchall())
df2.columns = [x[0] for x in cur.description]
print(len(df))
df2.head()

279


,TeamName,HomeTeam,FTR,Date
0,Nurnberg,Nurnberg,A,2012-03-31
1,Stuttgart,Stuttgart,A,2011-12-11
2,Wolfsburg,Wolfsburg,A,2011-08-13
3,Mainz,Mainz,H,2011-11-27
4,Freiburg,Freiburg,D,2012-02-18


In [147]:
df2.head()

,TeamName,HomeTeam,FTR,Date
0,Nurnberg,Nurnberg,A,2012-03-31
1,Stuttgart,Stuttgart,A,2011-12-11
2,Wolfsburg,Wolfsburg,A,2011-08-13
3,Mainz,Mainz,H,2011-11-27
4,Freiburg,Freiburg,D,2012-02-18


In [131]:
# How can I sort out the winning team?
#df2['winner'] = df2['TeamName'].apply(lambda x: 'W' if x == df2.HomeTeam & df2.FTR == "H" else "L" )


In [141]:
# This didn't seem to work.
df2.Winner = df2[(df2.TeamName == df2.HomeTeam) & (df2.FTR == "H")]
df2.head()

,TeamName,HomeTeam,FTR,Date
0,Nurnberg,Nurnberg,A,2012-03-31
1,Stuttgart,Stuttgart,A,2011-12-11
2,Wolfsburg,Wolfsburg,A,2011-08-13
3,Mainz,Mainz,H,2011-11-27
4,Freiburg,Freiburg,D,2012-02-18


In [143]:
#Make a new datafram with just the winners.
df3 = df2[(df2['TeamName'] == df2['HomeTeam']) & (df2['FTR'] == "H")]
print(len(df3))
df3.head()

# Why isn't this half of the games?

181


,TeamName,HomeTeam,FTR,Date
3,Mainz,Mainz,H,2011-11-27
5,M'gladbach,M'gladbach,H,2012-01-20
9,Hannover,Hannover,H,2011-10-23
11,Leverkusen,Leverkusen,H,2012-03-03
16,Dortmund,Dortmund,H,2012-04-11


In [151]:
# AGAIN.... Try to play with the data stats - team wins?
cur.execute("""SELECT TeamName, HomeTeam, AwayTeam, FTR, Date
               FROM Teams_in_Matches
               JOIN Unique_Teams
               USING(Unique_Team_ID)
               JOIN Matches
               USING(Match_ID)
               WHERE Season = 2011
               GROUP BY Match_ID
                
               ;""")
import pandas as pd
df5 = pd.DataFrame(cur.fetchall())
df5.columns = [x[0] for x in cur.description]
print(len(df))
df5.head()

992


,TeamName,HomeTeam,AwayTeam,FTR,Date
0,Nurnberg,Nurnberg,Bayern Munich,A,2012-03-31
1,Stuttgart,Stuttgart,Bayern Munich,A,2011-12-11
2,Wolfsburg,Wolfsburg,Bayern Munich,A,2011-08-13
3,Mainz,Mainz,Bayern Munich,H,2011-11-27
4,Freiburg,Freiburg,Bayern Munich,D,2012-02-18


In [153]:
#Make a new datafram with just the losers.
df4 = df5[(df5['TeamName'] == df5['AwayTeam']) & (df5['FTR'] == "A")]
print(len(df4))
df4.head()

102


,TeamName,HomeTeam,AwayTeam,FTR,Date
30,Dortmund,Bayern Munich,Dortmund,A,2011-11-19
35,Dresden,Bochum,Dresden,A,2012-03-02
46,Dresden,Aachen,Dresden,A,2012-03-30
48,Dresden,Braunschweig,Dresden,A,2011-10-22
65,Duisburg,Paderborn,Duisburg,A,2012-03-25


In [154]:
#Make a new datafram with just the losers.
df6 = df5[(df5['TeamName'] == df5['AwayTeam']) & (df5['FTR'] == "D")]
print(len(df6))
df6.head()

125


,TeamName,HomeTeam,AwayTeam,FTR,Date
34,Dresden,Hansa Rostock,Dresden,D,2011-12-18
37,Dresden,Paderborn,Dresden,D,2012-02-17
53,Duisburg,Cottbus,Duisburg,D,2011-12-18
61,Duisburg,Ingolstadt,Duisburg,D,2012-04-01
63,Duisburg,Braunschweig,Duisburg,D,2012-04-20


In [44]:
# I think we are going to have to reformat the date. Since it should be like [YYYY]-[MM]-[DD]
df.Date.head()

0    2012-05-13
1    2012-05-13
2    2012-05-13
3    2012-05-13
4    2012-05-13
Name: Date, dtype: object

## DarkSky API Weather

In [ ]:
# Location of Berlin, Germany (two sites)

# Latitude and longitude decimal coordinates are: 52.520008, 13.404954.

# Coordinates of Berlin in decimal degrees:  Latitude: 52.5243700°, Longitude: 13.4105300°

# Time zone in Berlin, Germany (GMT+1) - Central European Standard Time

Time Machine Request
          https://api.darksky.net/forecast/[key]/[latitude],[longitude],[time]
        
A Time Machine Request returns the observed (in the past) or forecasted (in the future) hour-by-hour weather and daily weather conditions for a particular date. A Time Machine request is identical in structure to a Forecast Request, except:

The currently data point will refer to the time provided, rather than the current time.
The minutely data block will be omitted, unless you are requesting a time within an hour of the present.
The hourly data block will contain data points starting at midnight (local time) of the day requested, and continuing until midnight (local time) of the following day.
The daily data block will contain a single data point referring to the requested date.
The alerts data block will be omitted.

In [ ]:
# Notes on Time Machine Request
***key required
# Your Dark Sky secret key. (Your secret key must be kept secret; in particular, do not embed it in JavaScript source code that you transmit to clients.)
***latitude required
# The latitude of a location (in decimal degrees). Positive is north, negative is south.
***longitude required
# The longitude of a location (in decimal degrees). Positive is east, negative is west.
***time required
# Either be a UNIX time (that is, seconds since midnight GMT on 1 Jan 1970) or a string formatted as follows: [YYYY]-[MM]-[DD]T[HH]:[MM]:[SS][timezone]. timezone should either be omitted (to refer to local time for the location being requested), Z (referring to GMT time), or +[HH][MM] or -[HH][MM] for an offset from GMT in hours and minutes. The timezone is only used for determining the time of the request; the response will always be relative to the local time zone.

In [ ]:
# Do I need to create a blank page with my secret name again?
import json

def get_keys(path):
    with open(path) as f:
        return json.load(f)

key = get_keys("/Users/whipxach/.secret/dark_sky_api.json")

api_key = key['api_key']

In [ ]:
# Try to make a request before setting up the 'class'
import requests
url = https://api.darksky.net/forecast/[api_key]/[latitude],[longitude],[time]

header = {"Authorization" : "Bearer {}".format(api_key)}
response = requests.get(url, header=header)
print(response)

In [ ]:
class WeatherGetter():
    pass

## Calculate summary statistics

## Histogram

## Load the data into MongoDB

## Refactor 

Refactor your code into functions or classes to make your code more modular, reusable, understandable, and maintainable!

In short--do what you need to do to get each separate piece of functionality working, and then refactor it into a class after you've figured it out!